## 9. Make sure you have copied the workspace archive (.zip) from the aggregator node to the collaborator node.

In [2]:
!pip install paramiko scp

from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient
from pathlib import Path
from getpass import getpass
afqdn = 'nnlicv431.inn.intel.com'
workspace_src = '/home/itrushkin/workspace'
archive_name = 'workspace.zip'
signed_cert = 'agg_to_col_two_signed_cert.zip'
key = 'cert/client/col_two.key'  
folder_name = archive_name.split('.')[0]
workspace_dst = '/home/itrushkin'
src_user = 'itrushki'


ssh_client = SSHClient()
ssh_client.set_missing_host_key_policy(AutoAddPolicy())
ssh_client.connect(afqdn, username=src_user, password=getpass(f'Enter {src_user}@{afqdn} password:'))
scp = SCPClient(ssh_client.get_transport())
scp.get(str(Path(workspace_src) / archive_name), workspace_dst)

Enter itrushki@nnlicv431.inn.intel.com password:········


# 11. Import the workspace archive.

In [13]:
%cd $workspace_dst
!rm -rf $folder_name
!fx workspace import --archive $archive_name
%cd $folder_name

/home/itrushkin
Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:38:26.404017: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:38:26.404043: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Workspace workspace.zip has been imported.
You may need to copy your PKI certificates to join the federation.

 ✔️ OK
/home/itrushkin/workspace


## 13. Create a collaborator certificate request.

In [14]:
!fx collaborator generate-cert-request -n two -d 2
scp.put('col_two_to_agg_cert_request.zip', workspace_src)

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:38:34.339652: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:38:34.339679: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=two, SAN=DNS:two
  Moving COLLABORATOR certificate to: cert/col_two
Archive col_two_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


# Get signed certificates from the aggregator

In [3]:
scp.get(str(Path(workspace_src) / 'agg_to_col_two_signed_cert.zip'), '.')
!fx collaborator certify --import agg_to_col_two_signed_cert.zip

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:40:47.402441: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:40:47.402467: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Certificate col_two.crt installed to PKI directory

 ✔️ OK


# Choose a device

In [4]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


# Run the collaborator process

In [8]:
!fx collaborator start -n two

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:41:39.484487: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:41:39.484515: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[18:41:40] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1607096500.4611084-902181;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1607096500.462991-506550;file:///home/itrushkin/.virtualenvs/fledge/li

[18:41:42] INFO     Histology > X_train Shape : (1999, 3, 150, 150)                                                                       ]8;id=1607096502.079867-678233;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:136
           INFO     Histology > y_train Shape : (1999,)                                                                                   ]8;id=1607096502.0812583-9454;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:137
           INFO     Histology > Train Samples : 1999                                                                                      ]8;id=1607096502.0824928-54680;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:138
           INFO     Histology > Valid Samples : 499                                                                                       ]8;id=1607096502.0838022-981723;file:///home/itrushkin/workspace/code/histology_utils.py

In [7]:
%cd /home/itrushkin/workspace

/home/itrushkin/workspace
